# Lösa övningsuppgifter i python

Då en del har hört av sig med förfrågningar om hur man går från teorin i boken till lösningar i kod kommer här ett exempel från veckans kapitel i boken, uppgift 322:

#### Övningsuppgift 322

Varje löradag klicka 10.00 ringer Fredrika sin pojkvän Fredrik. Men ofta svarar han tyvärr inte. Fredrikas frevensstudier visar att detta i genomsnitt inträffar fyra gånger av tio.   
Slummässigt väljer vi tolv gånger då Fredrika ringer till Fredrik.  
**Vad är sannolikheten att han inte svarar vid åtminstone 5 av dessa tillfällen?**

#### Lösning
Skriv upp kända värden:
* $n = 12$ (antal försök)
* $p = 0.4$ (sannolikhet för att Fredrik **inte** svarar)
* $q = 1 - p = 0.6$ (sannolikhet för att Fredrik svarar)

Vi vill veta sannolikheten att Fredrik inte svarar vid åtminstone 5 av dessa tillfällen. Detta kan vi skriva som $P(X \geq 5)$, där $X$ är antalet gånger Fredrik inte svarar.

In [73]:
n = 12
p = 0.4
q = 1 - p
x = 5

Identifiera vilken sannolikhetsfördelning som beskriver problemet.  
Vi har en kjedja av oberoende försök, där varje försök har två möjliga utfall. Detta är en **binomialfördelning**.

Här nedan ser vi ett exempel på hur utfallen i en binomialfördelning kan se ut.

In [74]:
import plot_tools as pt
import matplotlib.pyplot as plt

# Set parameters
num_paths = 6  # Number of paths to simulate and animate
p_success = 0.6  # Probability of success
tree_depth = 4 # Depth of the tree

animation = pt.generate_probability_tree_animation(num_paths, p_success, tree_depth, figsize=(8, 5))
plt.clf()
animation

<Figure size 576x360 with 0 Axes>

Binomialfördelningen definieras av två parametrar, $n$ och $p$.
$$Bin(n, p) = \binom{n}{x}p^x(1-p)^{n-x}$$

Då $n=12$ och $p=0.4$ blir vår sannolikhetsfördelning och vi är intresserade av $P(X \geq 5)$:
$$P(X \geq 5) = \sum_{x=5}^{12} \binom{12}{x}0.4^x(1-0.4)^{12-x}$$


In [75]:
import math
# Binomial coefficient calculator
def binomial_coefficient(n, k):
    return math.factorial(n) / (math.factorial(k) * math.factorial(n - k))

# Binomial probability calculator
def binomial_probability(n, k, p):
    return binomial_coefficient(n, k) * p**k * (1 - p)**(n - k)


# Cumulative binomial probability 
bin_cumulative = 0

# Calculate cumulative binomial probability
for k in range(x, n+1):
    bin_cumulative += binomial_probability(n, k, p)

print(f"Sannolikheten för att minst {x} av {n} telefonsamtal inte besvaras av Fredrik är: {bin_cumulative:.4f}")
# bin_cumulative

Sannolikheten för att minst 5 av 12 telefonsamtal inte besvaras av Fredrik är: 0.5618


Vi har alltså nått fram till svaret $P(X \geq 5) = 0.5618$.  

Observera dock att vi var tvugna att beräkna alla värden från $x=5$ till $x=12$ för att få fram detta svar. Detta är ganska jobbigt att göra...  
Därför kan vi använda oss av komplmentet till $P(X \geq 5)$, vilket är $P(X < 5)$, och därmed förenkla uppgiften till
$$
\begin{align*}
P(X \geq 5) &= 1 - P(X < 5) \\
&= 1 - (P(X=0) + P(X=1) + P(X=2) + P(X=3) + P(X=4)) \\
&= 1 - \sum_{x=0}^{4} \binom{12}{x}0.4^x(1-0.4)^{12-x}
\end{align*}
$$

Om vi beräknar $P(X < 5) = P(X \leq 4)$ får vi:

In [76]:
bin_cumulative_at_most = 0
for k in range(0, x):
    bin_cumulative_at_most += binomial_probability(n, k, p)

print(f"Sannolikheten för att högst {x-1} av {n} telefonsamtal inte besvaras av Fredrik är: {bin_cumulative_at_most:.4f}")

Sannolikheten för att högst 4 av 12 telefonsamtal inte besvaras av Fredrik är: 0.4382


Således blir $P(X \geq 5) = 1 - P(X \leq 4) = 1 - 0.4382 = 0.5618$.

In [77]:
print(f"Sannolikheten för att minst {x} av {n} telefonsamtal inte besvaras av Fredrik (1 - P(X <5)) är: {1-bin_cumulative_at_most:.4f}")

Sannolikheten för att minst 5 av 12 telefonsamtal inte besvaras av Fredrik (1 - P(X <5)) är: 0.5618


#### Poisson approximation
När antalet försök $n$ är stort och sannolikheten $p$ är liten kan vi approximera en binomialfördelning med en poissonfördelning:
$$ Po(\lambda) = \frac{e^{-\lambda}\lambda^x}{x!}$$

Vi kan testa detta i vårt exempel.

Vi har $n=12$ och $p=0.4$, vilket ger $\lambda = np = 12 \cdot 0.4 = 4.8$.

Vi kan nu beräkna $P(X \geq 5)$ med hjälp av poissonfördelningen:
$$
\begin{align*}
P(X \geq 5) &= 1 - P(X < 5) \\
&= 1 - (P(X=0) + P(X=1) + P(X=2) + P(X=3) + P(X=4)) \\
&= 1 - \sum_{x=0}^{4} \frac{e^{-\lambda}\lambda^x}{x!}
\end{align*}
$$

Vi implementerar därför poisson fördelningen i python:

In [78]:
def poisson_probability(lam, k):
    return math.exp(-lam) * lam**k / math.factorial(k)

def poisson_cumulative(lam, k):
    cumulative = 0
    for i in range(0, k):
        cumulative += poisson_probability(lam, i)
    return cumulative

Vi sätter $\lambda = 4.8$ och beräknar $P(X \geq 5)$:

In [79]:
n = 12
p = 0.4
x = 5

lam = n * p
poisson_approx = 1 - poisson_cumulative(lam, x)
poisson_error = abs(poisson_approx - bin_cumulative)

print(f"Sannolikheten för att minst {x} av {n} telefonsamtal inte besvaras av Fredrik är: {poisson_approx:.4f}")
print(f"Felet i approximationen är: {poisson_error:.4f}")

Sannolikheten för att minst 5 av 12 telefonsamtal inte besvaras av Fredrik är: 0.5237
Felet i approximationen är: 0.0381


Därmed ser vi att poisson approximationen ger oss ett ganska bra svar på frågan.
Felet i approximationen är 
$$0.5618 - 0.5127 = 0.0381$$


##### Minskar sannolikheten
Om vi låter $n$ vara oförändrar men minskar sannolikheten för att Fredrik inte svarar till, $p=0.1$
$$\lambda = np = 12 \cdot 0.1 = 1.2$$
och intresserar oss av $P(X \geq 2)$. _(Vi minskar $x$ då det nu är mindre sannolikt att Fredrik inte svarar)_

In [80]:
def binomial_cumulative(n, k, p):
    cumulative = 0
    for i in range(k):
        cumulative += binomial_probability(n, i, p)
    return cumulative

n = 12
p = 0.1
x = 2

binomial = 1 - binomial_cumulative(n, x, p)
poisson = 1 - poisson_cumulative(n * p, x)

poisson_error = abs(binomial - poisson)

print(f"Sannolikheten för att minst {x} av {n} telefonsamtal inte besvaras av Fredrik är: {binomial:.6f}")
print(f"Sannolikheten för att minst {x} av {n} telefonsamtal inte besvaras av Fredrik är: {poisson:.6f}")

print(f"Felet i approximationen är: {poisson_error:.6f}")

Sannolikheten för att minst 2 av 12 telefonsamtal inte besvaras av Fredrik är: 0.340998
Sannolikheten för att minst 2 av 12 telefonsamtal inte besvaras av Fredrik är: 0.337373
Felet i approximationen är: 0.003625


Nu ser vi att felet i approximationen minskat till $0.0036$.

I boken finns i tummregel som säger att om:  
$n \geq 10 \text{ och } p \leq 0.1$  
så fungerar poisson approximationen bra.